# スプラトゥーン２：勝利に貢献する要因の分析

## ライブラリの読み込み

In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

## データの読み込み

In [2]:
# プレイデータの読み込み
data = pd.read_csv("ikaWidgetCSV.csv", dtype=str, low_memory=False)

data.head()

,no,startTime,elapsedTime,win,udemae,udemaeName,udemaeIsX,udemaeIsReached,sPlusNumber,xRanking,...,bravo4ShoesSkillID,bravo4ShoesSkillName,bravo4ShoesSub1SkillID,bravo4ShoesSub1SkillName,bravo4ShoesSub2SkillID,bravo4ShoesSub2SkillName,bravo4ShoesSub3SkillID,bravo4ShoesSub3SkillName,Unnamed: 637,Unnamed: 638
0,16146,2021/02/14 17:05:14,312,1,128,X,1,1,0,0,...,5,スペシャル増加量アップ,5,スペシャル増加量アップ,9,スーパージャンプ時間短縮,200,爆風ダメージ軽減・改,NaN,NaN
1,16145,2021/02/14 16:21:14,330,1,128,X,1,1,0,0,...,111,受け身術,9,スーパージャンプ時間短縮,11,相手インク影響軽減,6,スペシャル減少量ダウン,NaN,NaN
2,16144,2021/02/14 16:15:14,317,0,19,S+,0,0,9,0,...,109,ステルスジャンプ,4,イカダッシュ速度アップ,4,イカダッシュ速度アップ,4,イカダッシュ速度アップ,NaN,NaN
3,16143,2021/02/14 16:08:14,303,1,19,S+,0,0,9,0,...,109,ステルスジャンプ,4,イカダッシュ速度アップ,4,イカダッシュ速度アップ,4,イカダッシュ速度アップ,NaN,NaN
4,16142,2021/02/14 16:01:14,320,1,19,S+,0,0,9,0,...,110,対物攻撃力アップ,200,爆風ダメージ軽減・改,9,スーパージャンプ時間短縮,11,相手インク影響軽減,NaN,NaN


## データの下準備

In [3]:
# データに含まれる列名を確認
list(data.columns)

['no',
 'startTime',
 'elapsedTime',
 'win',
 'udemae',
 'udemaeName',
 'udemaeIsX',
 'udemaeIsReached',
 'sPlusNumber',
 'xRanking',
 'xPower',
 'myCount',
 'otherCount',
 'winMeter',
 'weaponPaintPoint',
 'leaguePoint',
 'leagueMaxPoint',
 'leagueTeamEstimatePoint',
 'leagueOtherEstimatePoint',
 'gachiEstimatePower',
 'gachiEstimateXPower',
 'fesPoint',
 'fesPower',
 'fesMaxPower',
 'fesTeamEstimatePower',
 'fesOtherEstimatePower',
 'stageID',
 'stageName',
 'gameKey',
 'gameMode',
 'gameModeKey',
 'gameRule',
 'gameRuleKey',
 'gameType',
 'fesID',
 'fesMyTheme',
 'fesMyThemeKey',
 'fesOtherTheme',
 'fesOtherThemeKey',
 'fesMyColorR',
 'fesMyColorG',
 'fesMyColorB',
 'fesOtherColorR',
 'fesOtherColorG',
 'fesOtherColorB',
 'playerName',
 'playerPrincipalID',
 'playerRank',
 'playerUdemae',
 'playerudemaeName',
 'playerSPlusNumber',
 'playerUdemaeIsX',
 'playerUdemaeIsReached',
 'playerIsCrown',
 'playerStarRank',
 'playerKill',
 'playerDeath',
 'playerAssist',
 'playerSpecial',
 'pla

"Unnamed: 637", "Unnamed: 638"という不明の列があるので、確認する

In [4]:
data[["Unnamed: 637", "Unnamed: 638"]].describe()

,Unnamed: 637,Unnamed: 638
count,21,1
unique,11,1
top,ヒト移動速度アップ,スーパージャンプ時間短縮
freq,4,1


In [5]:
data.describe()

,no,startTime,elapsedTime,win,udemae,udemaeName,udemaeIsX,udemaeIsReached,sPlusNumber,xRanking,...,bravo4ShoesSkillID,bravo4ShoesSkillName,bravo4ShoesSub1SkillID,bravo4ShoesSub1SkillName,bravo4ShoesSub2SkillID,bravo4ShoesSub2SkillName,bravo4ShoesSub3SkillID,bravo4ShoesSub3SkillName,Unnamed: 637,Unnamed: 638
count,2486,2486,2486,2486,2486,2486,2486,2486,2486,2486,...,2486,2486,2486,2486,2486,2486,2486,2486,21,1
unique,2486,2486,301,2,17,8,2,2,10,6,...,29,30,29,28,27,29,29,28,11,1
top,16146,2021/02/14 17:05:14,0,1,0,S+,0,0,0,0,...,4,イカダッシュ速度アップ,4,イカダッシュ速度アップ,4,イカダッシュ速度アップ,4,イカダッシュ速度アップ,ヒト移動速度アップ,スーパージャンプ時間短縮
freq,1,1,591,1320,591,1321,2344,2344,1554,2481,...,403,403,465,465,426,426,382,382,4,1


"Unnamed: 637"列、"Unnamed: 638"列に値を持つ行数が、元のデータの数(2486)に比べて極端に少ない。

これらの列に値をもつデータを調べる

In [6]:
# 全ての列を表示できるように設定
pd.set_option('display.max_columns', None)

# Unnamed: 637 または Unnamed: 638 に NaN ではない値がある行を抽出
filtered_data = data[(data["Unnamed: 637"].notna()) | (data["Unnamed: 638"].notna())]

# 結果を表示
filtered_data

,no,startTime,elapsedTime,win,udemae,udemaeName,udemaeIsX,udemaeIsReached,sPlusNumber,xRanking,xPower,myCount,otherCount,winMeter,weaponPaintPoint,leaguePoint,leagueMaxPoint,leagueTeamEstimatePoint,leagueOtherEstimatePoint,gachiEstimatePower,gachiEstimateXPower,fesPoint,fesPower,fesMaxPower,fesTeamEstimatePower,fesOtherEstimatePower,stageID,stageName,gameKey,gameMode,gameModeKey,gameRule,gameRuleKey,gameType,fesID,fesMyTheme,fesMyThemeKey,fesOtherTheme,fesOtherThemeKey,fesMyColorR,fesMyColorG,fesMyColorB,fesOtherColorR,fesOtherColorG,fesOtherColorB,playerName,playerPrincipalID,playerRank,playerUdemae,playerudemaeName,playerSPlusNumber,playerUdemaeIsX,playerUdemaeIsReached,playerIsCrown,playerStarRank,playerKill,playerDeath,playerAssist,playerSpecial,playerRatio,playerAllKill,playerSortScore,playerPaintPoint,playerFesGrade,playerFesGradeName,playerWeaponID,playerWeaponName,playerWeaponSpecialID,playerWeaponSpecialName,playerWeaponSubID,playerWeaponSubName,playerHeadID,playerHeadName,playerHeadRarity,playerHeadKind,playerHeadBrandID,playerHeadBrandName,playerHeadBrandFrequentSkillID,playerHeadBrandFrequentSkillName,playerHeadSkillID,playerHeadSkillName,playerHeadSub1SkillID,playerHeadSub1SkillName,playerHeadSub2SkillID,playerHeadSub2SkillName,playerHeadSub3SkillID,playerHeadSub3SkillName,playerClothesID,playerClothesName,playerClothesRarity,playerClothesKind,playerClothesBrandID,playerClothesBrandName,playerClothesBrandFrequentSkillID,playerClothesBrandFrequentSkillName,playerClothesSkillID,playerClothesSkillName,playerClothesSub1SkillID,playerClothesSub1SkillName,playerClothesSub2SkillID,playerClothesSub2SkillName,playerClothesSub3SkillID,playerClothesSub3SkillName,playerShoesID,playerShoesName,playerShoesRarity,playerShoesKind,playerShoesBrandID,playerShoesBrandName,playerShoesBrandFrequentSkillID,playerShoesBrandFrequentSkillName,playerShoesSkillID,playerShoesSkillName,playerShoesSub1SkillID,playerShoesSub1SkillName,playerShoesSub2SkillID,playerShoesSub2SkillName,playerShoesSub3SkillID,playerShoesSub3SkillName,alpha1Name,alpha1PrincipalID,alpha1Rank,alpha1Udemae,alpha1udemaeName,alpha1SPlusNumber,alpha1UdemaeIsX,alpha1UdemaeIsReached,alpha1IsCrown,alpha1StarRank,alpha1Kill,alpha1Death,alpha1Assist,alpha1Special,alpha1Ratio,alpha1AllKill,alpha1SortScore,alpha1PaintPoint,alpha1FesGrade,alpha1FesGradeName,alpha1WeaponID,alpha1WeaponName,alpha1WeaponSpecialID,alpha1WeaponSpecialName,alpha1WeaponSubID,alpha1WeaponSubName,alpha1HeadID,alpha1HeadName,alpha1HeadRarity,alpha1HeadKind,alpha1HeadBrandID,alpha1HeadBrandName,alpha1HeadBrandFrequentSkillID,alpha1HeadBrandFrequentSkillName,alpha1HeadSkillID,alpha1HeadSkillName,alpha1HeadSub1SkillID,alpha1HeadSub1SkillName,alpha1HeadSub2SkillID,alpha1HeadSub2SkillName,alpha1HeadSub3SkillID,alpha1HeadSub3SkillName,alpha1ClothesID,alpha1ClothesName,alpha1ClothesRarity,alpha1ClothesKind,alpha1ClothesBrandID,alpha1ClothesBrandName,alpha1ClothesBrandFrequentSkillID,alpha1ClothesBrandFrequentSkillName,alpha1ClothesSkillID,alpha1ClothesSkillName,alpha1ClothesSub1SkillID,alpha1ClothesSub1SkillName,alpha1ClothesSub2SkillID,alpha1ClothesSub2SkillName,alpha1ClothesSub3SkillID,alpha1ClothesSub3SkillName,alpha1ShoesID,alpha1ShoesName,alpha1ShoesRarity,alpha1ShoesKind,alpha1ShoesBrandID,alpha1ShoesBrandName,alpha1ShoesBrandFrequentSkillID,alpha1ShoesBrandFrequentSkillName,alpha1ShoesSkillID,alpha1ShoesSkillName,alpha1ShoesSub1SkillID,alpha1ShoesSub1SkillName,alpha1ShoesSub2SkillID,alpha1ShoesSub2SkillName,alpha1ShoesSub3SkillID,alpha1ShoesSub3SkillName,alpha2Name,alpha2PrincipalID,alpha2Rank,alpha2Udemae,alpha2udemaeName,alpha2SPlusNumber,alpha2UdemaeIsX,alpha2UdemaeIsReached,alpha2IsCrown,alpha2StarRank,alpha2Kill,alpha2Death,alpha2Assist,alpha2Special,alpha2Ratio,alpha2AllKill,alpha2SortScore,alpha2PaintPoint,alpha2FesGrade,alpha2FesGradeName,alpha2WeaponID,alpha2WeaponName,alpha2WeaponSpecialID,alpha2WeaponSpecialName,alpha2WeaponSubID,alpha2WeaponSubName,alpha2HeadID,alpha2HeadName,alpha2HeadRarity,al

In [7]:
# 表示したい列を選択
selected_columns = ["alpha1Name", "alpha1PrincipalID",
                    "alpha2Name", "alpha2PrincipalID",
                    "alpha3Name","alpha3PrincipalID",
                    "bravo1Name", "bravo1PrincipalID",
                    "bravo2Name", "bravo2PrincipalID",
                    "bravo3Name", "bravo3PrincipalID",
                    "bravo4Name", "bravo4PrincipalID"]

# 結果を表示
filtered_data[selected_columns]

,alpha1Name,alpha1PrincipalID,alpha2Name,alpha2PrincipalID,alpha3Name,alpha3PrincipalID,bravo1Name,bravo1PrincipalID,bravo2Name,bravo2PrincipalID,bravo3Name,bravo3PrincipalID,bravo4Name,bravo4PrincipalID
229,"""アイアンマン2 """,ec377872c652460c,"""まほうつかい イカ""",a39c12b6124c81da,"""へたこいか""",f3f8cd8e091a4a45,"""したりくん""",be898af1ee9cef28,"""No","75""",相手インク影響軽減,"""ふ""",イカダッシュ速度アップ,"""エダ"""
241,"""たまきち""",33dc83105b84b91e,"""コクリア""",7b7f7672f55c2bf6,"""めかぶのせんぱんつ""",f85f8d1313681867,"""†aluhu†""",d83b0ee6db176f2b,"""(⌒","_ゝ⌒)""",はてな,"""はなたば""",スペシャル性能アップ,"""ENDer"""
479,"""なめこ""",0ad858f70234749a,"""ゆ""",a34ca6a13d408438,"""ロストシップ""",75446769ec3da2c6,"""humeimei9""",c26cb075ff26b9c5,"""ちょむすけ""",f64e7478465cf57b,"""TI","*KANNA*""",インク効率アップ(メイン),"""なまざかな"""
547,"""M","T """,インク効率アップ(サブ),"""TT's ういづき""",イカダッシュ速度アップ,"""みるくちょこ*ゝ*""",サブ性能アップ,"""タスク""",ヒト移動速度アップ,"""ミスターすまない☆""",インク効率アップ(メイン),"""konoha_3""",イカダッシュ速度アップ,"""りょうすけ"""
808,""" ゆ り * """,d30a1ca6b9a1adf1,"""おならtank""",789a7fc03907b794,"""あおもり たろう""",f702eb43bc19668a,"""Z","Hikoki""",イカダッシュ速度アップ,"""モモペン¾""",イカダッシュ速度アップ,"""だいこん""",インク効率アップ(メイン),"""yuki*くーげる*"""
904,"""Minomushi""",f6f9589cfdfbdcd9,"""~`;:""'",".*/""",インク回復力アップ,"""ドスコイのももにく""",はてな,"""よ～いドン！！""",サブ性能アップ,"""よろずや""",爆風ダメージ軽減・改,"""こふゆるり""",メイン性能アップ,"""みりょー"""
958,"""muu""",7fc845424a757286,"""さとう""",05568be377dd6d09,"""BELLE""",dad3b5af6d2f8c18,"""Ms","Masako""",はてな,"""シフォン""",インク回復力アップ,"""オーノ""",スペシャル減少量ダウン,"""ひろむくん"""
959,"""Ms","Masako""",はてな,"""た""",イカダッシュ速度アップ,"""ツバメ""",スーパージャンプ時間短縮,"""muu""",はてな,"""さとう""",インク効率アップ(メイン),"""こころ""",スペシャル減少量ダウン,"""PILOT"""
960,"""た""",1ef799316a28d744,"""ぬおおおおおおおおお""",3a742c1bc0854ff4,"""こころ""",4ced4e952eb49894,"""Ms","Masako""",はてな,"""muu""",はてな,"""ちさみけ&かずきじ""",インク効率アップ(メイン),"""さとう"""
961,"""た""",1ef799316a28d744,"""おすし""",80af563c80dab958,"""さとう""",05568be377dd6d09,"""Ms","Masako""",はてな,"""ミシル""",ヒト移動速度アップ,"""muu""",はてな,"""ちさみけ&かずきじ"""


### 列のずれを修正

一部の行で**カラムが本来の構造からずれて記録されている**ことが確認された。
具体的には、プレイヤーの PrincipalID（プレイヤー識別ID）の列に、本来 Name（プレイヤー名）が入ってしまい、それ以降のカラムがすべて右にずれてしまっている。

**原因**
- PrincipalID には ランダムな英数字の識別ID が入るはずだが、その前の Name 列に'"'が含まれるなどの理由で、一部の行では PrincipalID の代わりにプレイヤー名が記録されている
- それにより、PrincipalID 列以降の値がすべて1つずつ右にずれてしまっている

**対処方法**
1. PrincipalID の値をチェックし、ランダムな英数字の形式でない場合は Name の値と統合する
2. PrincipalID を None にした後、その行のすべての値を左詰めする

In [8]:
# filtered_data のインデックスを取得
filtered_index = filtered_data.index

# ランダムな英数字（PrincipalID）のパターンを作成
id_pattern = re.compile(r"^[a-f0-9]{8,}$", re.IGNORECASE)

# PrincipalID列と、Name列の列名リストを作成
id_columns = ["playerPrincipalID", "alpha1PrincipalID", "alpha2PrincipalID", "alpha3PrincipalID",
              "bravo1PrincipalID", "bravo2PrincipalID", "bravo3PrincipalID", "bravo4PrincipalID"]

name_columns = ["playerName", "alpha1Name", "alpha2Name", "alpha3Name",
                "bravo1Name", "bravo2Name", "bravo3Name", "bravo4Name"]

In [9]:
def fix_principal_id(row):
    updated_row = row.copy()  # 元の行をコピー

    # IDをNameへ統合（プレイヤーごとに処理し、その都度左詰め）
    for player_type in ["player", "alpha1", "alpha2", "alpha3", "bravo1", "bravo2", "bravo3", "bravo4"]:
        id_col = player_type + "PrincipalID"
        name_col = player_type + "Name"

        id_value = str(updated_row[id_col]).strip()
        name_value = str(updated_row[name_col]).strip()

        # **IDがランダムな英数字でない場合は Name に統合**
        if not id_pattern.match(id_value):
            updated_row[name_col] = name_value + id_value  # `Name` に結合
            updated_row[id_col] = None  # `ID` を空欄にする

            # None以降の列を左詰め処理
            temp_values = [x for x in updated_row if x is not None]  # Noneを除く
            temp_values += [""] * (len(updated_row) - len(temp_values))  # 空白で埋める
            updated_row[:] = temp_values  # 値の並びだけ更新（カラムは維持）
            
    return updated_row


# データフレームに適用
cleaned_data = filtered_data.apply(fix_principal_id, axis=1)

# 結果を表示
cleaned_data.head()

,no,startTime,elapsedTime,win,udemae,udemaeName,udemaeIsX,udemaeIsReached,sPlusNumber,xRanking,xPower,myCount,otherCount,winMeter,weaponPaintPoint,leaguePoint,leagueMaxPoint,leagueTeamEstimatePoint,leagueOtherEstimatePoint,gachiEstimatePower,gachiEstimateXPower,fesPoint,fesPower,fesMaxPower,fesTeamEstimatePower,fesOtherEstimatePower,stageID,stageName,gameKey,gameMode,gameModeKey,gameRule,gameRuleKey,gameType,fesID,fesMyTheme,fesMyThemeKey,fesOtherTheme,fesOtherThemeKey,fesMyColorR,fesMyColorG,fesMyColorB,fesOtherColorR,fesOtherColorG,fesOtherColorB,playerName,playerPrincipalID,playerRank,playerUdemae,playerudemaeName,playerSPlusNumber,playerUdemaeIsX,playerUdemaeIsReached,playerIsCrown,playerStarRank,playerKill,playerDeath,playerAssist,playerSpecial,playerRatio,playerAllKill,playerSortScore,playerPaintPoint,playerFesGrade,playerFesGradeName,playerWeaponID,playerWeaponName,playerWeaponSpecialID,playerWeaponSpecialName,playerWeaponSubID,playerWeaponSubName,playerHeadID,playerHeadName,playerHeadRarity,playerHeadKind,playerHeadBrandID,playerHeadBrandName,playerHeadBrandFrequentSkillID,playerHeadBrandFrequentSkillName,playerHeadSkillID,playerHeadSkillName,playerHeadSub1SkillID,playerHeadSub1SkillName,playerHeadSub2SkillID,playerHeadSub2SkillName,playerHeadSub3SkillID,playerHeadSub3SkillName,playerClothesID,playerClothesName,playerClothesRarity,playerClothesKind,playerClothesBrandID,playerClothesBrandName,playerClothesBrandFrequentSkillID,playerClothesBrandFrequentSkillName,playerClothesSkillID,playerClothesSkillName,playerClothesSub1SkillID,playerClothesSub1SkillName,playerClothesSub2SkillID,playerClothesSub2SkillName,playerClothesSub3SkillID,playerClothesSub3SkillName,playerShoesID,playerShoesName,playerShoesRarity,playerShoesKind,playerShoesBrandID,playerShoesBrandName,playerShoesBrandFrequentSkillID,playerShoesBrandFrequentSkillName,playerShoesSkillID,playerShoesSkillName,playerShoesSub1SkillID,playerShoesSub1SkillName,playerShoesSub2SkillID,playerShoesSub2SkillName,playerShoesSub3SkillID,playerShoesSub3SkillName,alpha1Name,alpha1PrincipalID,alpha1Rank,alpha1Udemae,alpha1udemaeName,alpha1SPlusNumber,alpha1UdemaeIsX,alpha1UdemaeIsReached,alpha1IsCrown,alpha1StarRank,alpha1Kill,alpha1Death,alpha1Assist,alpha1Special,alpha1Ratio,alpha1AllKill,alpha1SortScore,alpha1PaintPoint,alpha1FesGrade,alpha1FesGradeName,alpha1WeaponID,alpha1WeaponName,alpha1WeaponSpecialID,alpha1WeaponSpecialName,alpha1WeaponSubID,alpha1WeaponSubName,alpha1HeadID,alpha1HeadName,alpha1HeadRarity,alpha1HeadKind,alpha1HeadBrandID,alpha1HeadBrandName,alpha1HeadBrandFrequentSkillID,alpha1HeadBrandFrequentSkillName,alpha1HeadSkillID,alpha1HeadSkillName,alpha1HeadSub1SkillID,alpha1HeadSub1SkillName,alpha1HeadSub2SkillID,alpha1HeadSub2SkillName,alpha1HeadSub3SkillID,alpha1HeadSub3SkillName,alpha1ClothesID,alpha1ClothesName,alpha1ClothesRarity,alpha1ClothesKind,alpha1ClothesBrandID,alpha1ClothesBrandName,alpha1ClothesBrandFrequentSkillID,alpha1ClothesBrandFrequentSkillName,alpha1ClothesSkillID,alpha1ClothesSkillName,alpha1ClothesSub1SkillID,alpha1ClothesSub1SkillName,alpha1ClothesSub2SkillID,alpha1ClothesSub2SkillName,alpha1ClothesSub3SkillID,alpha1ClothesSub3SkillName,alpha1ShoesID,alpha1ShoesName,alpha1ShoesRarity,alpha1ShoesKind,alpha1ShoesBrandID,alpha1ShoesBrandName,alpha1ShoesBrandFrequentSkillID,alpha1ShoesBrandFrequentSkillName,alpha1ShoesSkillID,alpha1ShoesSkillName,alpha1ShoesSub1SkillID,alpha1ShoesSub1SkillName,alpha1ShoesSub2SkillID,alpha1ShoesSub2SkillName,alpha1ShoesSub3SkillID,alpha1ShoesSub3SkillName,alpha2Name,alpha2PrincipalID,alpha2Rank,alpha2Udemae,alpha2udemaeName,alpha2SPlusNumber,alpha2UdemaeIsX,alpha2UdemaeIsReached,alpha2IsCrown,alpha2StarRank,alpha2Kill,alpha2Death,alpha2Assist,alpha2Special,alpha2Ratio,alpha2AllKill,alpha2SortScore,alpha2PaintPoint,alpha2FesGrade,alpha2FesGradeName,alpha2WeaponID,alpha2WeaponName,alpha2WeaponSpecialID,alpha2WeaponSpecialName,alpha2WeaponSubID,alpha2WeaponSubName,alpha2HeadID,alpha2HeadName,alpha2HeadRarity,al

In [10]:
cleaned_data[selected_columns]

,alpha1Name,alpha1PrincipalID,alpha2Name,alpha2PrincipalID,alpha3Name,alpha3PrincipalID,bravo1Name,bravo1PrincipalID,bravo2Name,bravo2PrincipalID,bravo3Name,bravo3PrincipalID,bravo4Name,bravo4PrincipalID
229,"""アイアンマン2 """,ec377872c652460c,"""まほうつかい イカ""",a39c12b6124c81da,"""へたこいか""",f3f8cd8e091a4a45,"""したりくん""",be898af1ee9cef28,"""No75""",87e802a4b4e9ca2d,"""ふ""",f6d8789a8c61b067,"""エダ""",d9679d907fad3efa
241,"""たまきち""",33dc83105b84b91e,"""コクリア""",7b7f7672f55c2bf6,"""めかぶのせんぱんつ""",f85f8d1313681867,"""†aluhu†""",d83b0ee6db176f2b,"""(⌒_ゝ⌒)""",24982de78f512cbc,"""はなたば""",e5d87344253b6e55,"""ENDer""",0832a4ae65c6393e
479,"""なめこ""",0ad858f70234749a,"""ゆ""",a34ca6a13d408438,"""ロストシップ""",75446769ec3da2c6,"""humeimei9""",c26cb075ff26b9c5,"""ちょむすけ""",f64e7478465cf57b,"""TI*KANNA*""",0e40e4ebb900603c,"""なまざかな""",dca92b6a51c6928b
547,"""MT """,d151242d38f7e3bd,"""TT's ういづき""",1924d6017041522c,"""みるくちょこ*ゝ*""",9b8753fb4bcf8d9b,"""タスク""",debcf290a0e82ba2,"""ミスターすまない☆""",c6b35a60ef28f59a,"""konoha_3""",fae7757da64ee0f9,"""りょうすけ""",fec33b1080a69684
808,""" ゆ り * """,d30a1ca6b9a1adf1,"""おならtank""",789a7fc03907b794,"""あおもり たろう""",f702eb43bc19668a,"""ZHikoki""",0b8c8332b12c7b1e,"""モモペン¾""",1adc4faa2080b155,"""だいこん""",e3d161405de572b1,"""yuki*くーげる*""",9e3d5d25bf5a5545
904,"""Minomushi""",f6f9589cfdfbdcd9,"""~`;:""'.*/""",ad83b711dfb11367,"""ドスコイのももにく""",21bc4e6e05aecd99,"""よ～いドン！！""",e260a2f5e894aff9,"""よろずや""",3d5de5876a978976,"""こふゆるり""",832729beeac22eda,"""みりょー""",65907ec84bda8c4e
958,"""muu""",7fc845424a757286,"""さとう""",05568be377dd6d09,"""BELLE""",dad3b5af6d2f8c18,"""MsMasako""",ef5aedbbfa2b9332,"""シフォン""",5bc5fd186fbd957e,"""オーノ""",0cfa52d8b786832f,"""ひろむくん""",2dc2479c9be80f52
959,"""MsMasako""",ef5aedbbfa2b9332,"""た""",1ef799316a28d744,"""ツバメ""",11b160772e8206c4,"""muu""",7fc845424a757286,"""さとう""",05568be377dd6d09,"""こころ""",4ced4e952eb49894,"""PILOT""",e44846b8ce8b7cfd
960,"""た""",1ef799316a28d744,"""ぬおおおおおおおおお""",3a742c1bc0854ff4,"""こころ""",4ced4e952eb49894,"""MsMasako""",ef5aedbbfa2b9332,"""muu""",7fc845424a757286,"""ちさみけ&かずきじ""",6870a6429be184b0,"""さとう""",05568be377dd6d09
961,"""た""",1ef799316a28d744,"""おすし""",80af563c80dab958,"""さとう""",05568be377dd6d09,"""MsMasako""",ef5aedbbfa2b9332,"""ミシル""",97ac65c9ef5d2bc2,"""muu""",7fc845424a757286,"""ちさみけ&かずきじ""",6870a6429be184b0


In [11]:
# data の間違いがあったデータを cleaned_data に置き換え
data.loc[filtered_index] = cleaned_data

# 修正後の data を確認
print(data.shape)

(2486, 639)


In [12]:
# 処理できてない行を一旦手動で削除

data = data[data["no"] != "9880"]

In [14]:
data.head()

,no,startTime,elapsedTime,win,udemae,udemaeName,udemaeIsX,udemaeIsReached,sPlusNumber,xRanking,xPower,myCount,otherCount,winMeter,weaponPaintPoint,leaguePoint,leagueMaxPoint,leagueTeamEstimatePoint,leagueOtherEstimatePoint,gachiEstimatePower,gachiEstimateXPower,fesPoint,fesPower,fesMaxPower,fesTeamEstimatePower,fesOtherEstimatePower,stageID,stageName,gameKey,gameMode,gameModeKey,gameRule,gameRuleKey,gameType,fesID,fesMyTheme,fesMyThemeKey,fesOtherTheme,fesOtherThemeKey,fesMyColorR,fesMyColorG,fesMyColorB,fesOtherColorR,fesOtherColorG,fesOtherColorB,playerName,playerPrincipalID,playerRank,playerUdemae,playerudemaeName,playerSPlusNumber,playerUdemaeIsX,playerUdemaeIsReached,playerIsCrown,playerStarRank,playerKill,playerDeath,playerAssist,playerSpecial,playerRatio,playerAllKill,playerSortScore,playerPaintPoint,playerFesGrade,playerFesGradeName,playerWeaponID,playerWeaponName,playerWeaponSpecialID,playerWeaponSpecialName,playerWeaponSubID,playerWeaponSubName,playerHeadID,playerHeadName,playerHeadRarity,playerHeadKind,playerHeadBrandID,playerHeadBrandName,playerHeadBrandFrequentSkillID,playerHeadBrandFrequentSkillName,playerHeadSkillID,playerHeadSkillName,playerHeadSub1SkillID,playerHeadSub1SkillName,playerHeadSub2SkillID,playerHeadSub2SkillName,playerHeadSub3SkillID,playerHeadSub3SkillName,playerClothesID,playerClothesName,playerClothesRarity,playerClothesKind,playerClothesBrandID,playerClothesBrandName,playerClothesBrandFrequentSkillID,playerClothesBrandFrequentSkillName,playerClothesSkillID,playerClothesSkillName,playerClothesSub1SkillID,playerClothesSub1SkillName,playerClothesSub2SkillID,playerClothesSub2SkillName,playerClothesSub3SkillID,playerClothesSub3SkillName,playerShoesID,playerShoesName,playerShoesRarity,playerShoesKind,playerShoesBrandID,playerShoesBrandName,playerShoesBrandFrequentSkillID,playerShoesBrandFrequentSkillName,playerShoesSkillID,playerShoesSkillName,playerShoesSub1SkillID,playerShoesSub1SkillName,playerShoesSub2SkillID,playerShoesSub2SkillName,playerShoesSub3SkillID,playerShoesSub3SkillName,alpha1Name,alpha1PrincipalID,alpha1Rank,alpha1Udemae,alpha1udemaeName,alpha1SPlusNumber,alpha1UdemaeIsX,alpha1UdemaeIsReached,alpha1IsCrown,alpha1StarRank,alpha1Kill,alpha1Death,alpha1Assist,alpha1Special,alpha1Ratio,alpha1AllKill,alpha1SortScore,alpha1PaintPoint,alpha1FesGrade,alpha1FesGradeName,alpha1WeaponID,alpha1WeaponName,alpha1WeaponSpecialID,alpha1WeaponSpecialName,alpha1WeaponSubID,alpha1WeaponSubName,alpha1HeadID,alpha1HeadName,alpha1HeadRarity,alpha1HeadKind,alpha1HeadBrandID,alpha1HeadBrandName,alpha1HeadBrandFrequentSkillID,alpha1HeadBrandFrequentSkillName,alpha1HeadSkillID,alpha1HeadSkillName,alpha1HeadSub1SkillID,alpha1HeadSub1SkillName,alpha1HeadSub2SkillID,alpha1HeadSub2SkillName,alpha1HeadSub3SkillID,alpha1HeadSub3SkillName,alpha1ClothesID,alpha1ClothesName,alpha1ClothesRarity,alpha1ClothesKind,alpha1ClothesBrandID,alpha1ClothesBrandName,alpha1ClothesBrandFrequentSkillID,alpha1ClothesBrandFrequentSkillName,alpha1ClothesSkillID,alpha1ClothesSkillName,alpha1ClothesSub1SkillID,alpha1ClothesSub1SkillName,alpha1ClothesSub2SkillID,alpha1ClothesSub2SkillName,alpha1ClothesSub3SkillID,alpha1ClothesSub3SkillName,alpha1ShoesID,alpha1ShoesName,alpha1ShoesRarity,alpha1ShoesKind,alpha1ShoesBrandID,alpha1ShoesBrandName,alpha1ShoesBrandFrequentSkillID,alpha1ShoesBrandFrequentSkillName,alpha1ShoesSkillID,alpha1ShoesSkillName,alpha1ShoesSub1SkillID,alpha1ShoesSub1SkillName,alpha1ShoesSub2SkillID,alpha1ShoesSub2SkillName,alpha1ShoesSub3SkillID,alpha1ShoesSub3SkillName,alpha2Name,alpha2PrincipalID,alpha2Rank,alpha2Udemae,alpha2udemaeName,alpha2SPlusNumber,alpha2UdemaeIsX,alpha2UdemaeIsReached,alpha2IsCrown,alpha2StarRank,alpha2Kill,alpha2Death,alpha2Assist,alpha2Special,alpha2Ratio,alpha2AllKill,alpha2SortScore,alpha2PaintPoint,alpha2FesGrade,alpha2FesGradeName,alpha2WeaponID,alpha2WeaponName,alpha2WeaponSpecialID,alpha2WeaponSpecialName,alpha2WeaponSubID,alpha2WeaponSubName,alpha2HeadID,alpha2HeadName,alpha2HeadRarity,al

### 不要な列の削除

In [15]:
# 使わないカラムを削除（例: 試合情報、武器・ギアデータ）
drop_columns = [
    'startTime', 'elapsedTime', 'udemae', 'udemaeIsX', 'udemaeIsReached', 'sPlusNumber',
    'xRanking', 'xPower', 'myCount', 'otherCount', 'winMeter', 'weaponPaintPoint', 'leaguePoint',
    'leagueMaxPoint', 'leagueTeamEstimatePoint', 'leagueOtherEstimatePoint', 'gachiEstimatePower',
    'gachiEstimateXPower', 'fesPoint', 'fesPower', 'fesMaxPower', 'fesTeamEstimatePower', 'fesOtherEstimatePower',
    'stageID', 'gameKey', 'gameMode', 'gameModeKey', 'gameRuleKey', 'gameType', 'fesID',
    'fesMyTheme', 'fesMyThemeKey', 'fesOtherTheme', 'fesOtherThemeKey', 'fesMyColorR', 'fesMyColorG',
    'fesMyColorB', 'fesOtherColorR', 'fesOtherColorG', 'fesOtherColorB', 
    'Name', 'udemaeName', 'SPlusNumber', 'UdemaeIsX', 'UdemaeIsReached',
    'IsCrown', 'StarRank', 'Ratio', 'AllKill', 'SortScore',
    'FesGrade', 'FesGradeName', 'WeaponID', 'WeaponName', 'WeaponSpecialID', 'WeaponSpecialName',
    'WeaponSubID', 'WeaponSubName', 'HeadID', 'HeadName', 'HeadRarity', 'HeadKind', 'HeadBrandID',
    'HeadBrandName', 'HeadBrandFrequentSkillID', 'HeadBrandFrequentSkillName', 'HeadSkillID', 'HeadSkillName',
    'HeadSub1SkillID', 'HeadSub1SkillName', 'HeadSub2SkillID', 'HeadSub2SkillName', 'HeadSub3SkillID', 'HeadSub3SkillName',
    'ClothesID', 'ClothesName', 'ClothesRarity', 'ClothesKind', 'ClothesBrandID', 'ClothesBrandName',
    'ClothesBrandFrequentSkillID', 'ClothesBrandFrequentSkillName', 'ClothesSkillID', 'ClothesSkillName',
    'ClothesSub1SkillID', 'ClothesSub1SkillName', 'ClothesSub2SkillID', 'ClothesSub2SkillName', 'ClothesSub3SkillID', 'ClothesSub3SkillName',
    'ShoesID', 'ShoesName', 'ShoesRarity', 'ShoesKind', 'ShoesBrandID', 'ShoesBrandName', 'ShoesBrandFrequentSkillID',
    'ShoesBrandFrequentSkillName', 'ShoesSkillID', 'ShoesSkillName', 'ShoesSub1SkillID', 'ShoesSub1SkillName',
    'ShoesSub2SkillID', 'ShoesSub2SkillName', 'ShoesSub3SkillID', 'ShoesSub3SkillName',
    'Unnamed: 637', 'Unnamed: 638', "PrincipalID"
]

# drop_columns に含まれる文字列を含むカラムを削除
columns_to_drop = [col for col in data.columns if any(drop in col for drop in drop_columns)]
data_dropped = data.drop(columns=columns_to_drop)

# 削除後のカラムを確認
print(list(data_dropped.columns))

['no', 'win', 'gameRule', 'playerRank', 'playerUdemae', 'playerKill', 'playerDeath', 'playerAssist', 'playerSpecial', 'playerPaintPoint', 'alpha1Rank', 'alpha1Udemae', 'alpha1Kill', 'alpha1Death', 'alpha1Assist', 'alpha1Special', 'alpha1PaintPoint', 'alpha2Rank', 'alpha2Udemae', 'alpha2Kill', 'alpha2Death', 'alpha2Assist', 'alpha2Special', 'alpha2PaintPoint', 'alpha3Rank', 'alpha3Udemae', 'alpha3Kill', 'alpha3Death', 'alpha3Assist', 'alpha3Special', 'alpha3PaintPoint', 'bravo1Rank', 'bravo1Udemae', 'bravo1Kill', 'bravo1Death', 'bravo1Assist', 'bravo1Special', 'bravo1PaintPoint', 'bravo2Rank', 'bravo2Udemae', 'bravo2Kill', 'bravo2Death', 'bravo2Assist', 'bravo2Special', 'bravo2PaintPoint', 'bravo3Rank', 'bravo3Udemae', 'bravo3Kill', 'bravo3Death', 'bravo3Assist', 'bravo3Special', 'bravo3PaintPoint', 'bravo4Rank', 'bravo4Udemae', 'bravo4Kill', 'bravo4Death', 'bravo4Assist', 'bravo4Special', 'bravo4PaintPoint']


### 縦長データに整形

**データの規則**

このデータセットは、**1試合ごとの情報が1行にまとまっている**形式。  
各プレイヤーの情報は、データを記録したプレイヤー（プレイヤーA）を基準に、以下のような命名規則で管理されている。

- **試合全体のデータ**（`no ~ fesOtherColorB`）：試合のルールやステージ情報
- **プレイヤーA**（`player`）：データを取得した本人
- **味方チーム**（`alpha1` ~ `alpha3`）
- **敵チーム**（`bravo1` ~ `bravo4`）

1行に **8人のプレイヤーのデータ** が含まれており、それぞれの列に武器、キル数、デス数などの情報が格納されている。

以下で、1行に1試合1プレイヤーの行をもつ形式に変換する。

In [16]:
# 列名をアンダースコアで区切る形式に変換
data_renamed = data_dropped.rename(columns=lambda x: x.replace("player", "player_")
                                    .replace("alpha1", "alpha1_")
                                    .replace("alpha2", "alpha2_")
                                    .replace("alpha3", "alpha3_")
                                    .replace("bravo1", "bravo1_")
                                    .replace("bravo2", "bravo2_")
                                    .replace("bravo3", "bravo3_")
                                    .replace("bravo4", "bravo4_"))

# 統一したい列名に共通して含まれる単語リスト
common_columns = ["Rank", "Udemae", "Kill", "Death", "Assist", "Special", "PaintPoint"]

# 縦長データに変換
id_vars = ["no", "win", "gameRule"]
value_vars = [col for col in data_renamed.columns if any(stub in col for stub in common_columns)]

melted_data = data_renamed.melt(id_vars=id_vars, value_vars=value_vars, var_name="player_stat", value_name="value")

# player_stat を player_type と stat に分割
melted_data[['player_type', 'stat']] = melted_data['player_stat'].str.extract(r'^(player|alpha\d|bravo\d)_(.+)$')

# データをpivot
long_data = melted_data.pivot(index=['no', 'win', 'gameRule', 'player_type'], columns='stat', values='value').reset_index()

# 結果を確認
long_data.head(20)

stat,no,win,gameRule,player_type,Assist,Death,Kill,PaintPoint,Rank,Special,Udemae
0,10075,0,ガチヤグラ,alpha1,2,11,3,905,28,2,-1
1,10075,0,ガチヤグラ,alpha2,2,7,8,912,22,3,-1
2,10075,0,ガチヤグラ,alpha3,2,10,6,1063,34,3,-1
3,10075,0,ガチヤグラ,bravo1,6,3,7,1173,1,6,-1
4,10075,0,ガチヤグラ,bravo2,0,5,8,888,75,2,-1
5,10075,0,ガチヤグラ,bravo3,4,9,10,736,57,2,-1
6,10075,0,ガチヤグラ,bravo4,1,4,9,1206,60,4,-1
7,10075,0,ガチヤグラ,player,2,7,3,1097,61,2,19
8,10076,0,ガチヤグラ,alpha1,2,5,9,1340,34,5,-1
9,10076,0,ガチヤグラ,alpha2,2,11,10,644,57,1,-1


勝利に貢献する一般的な要因を知ることが目的のため、player_typeがplayerのデータを削除し偏りをなくす。

In [17]:
long_data_dropped = long_data[long_data["player_type"] != "player"]

long_data_dropped.head(10)

stat,no,win,gameRule,player_type,Assist,Death,Kill,PaintPoint,Rank,Special,Udemae
0,10075,0,ガチヤグラ,alpha1,2,11,3,905,28,2,-1
1,10075,0,ガチヤグラ,alpha2,2,7,8,912,22,3,-1
2,10075,0,ガチヤグラ,alpha3,2,10,6,1063,34,3,-1
3,10075,0,ガチヤグラ,bravo1,6,3,7,1173,1,6,-1
4,10075,0,ガチヤグラ,bravo2,0,5,8,888,75,2,-1
5,10075,0,ガチヤグラ,bravo3,4,9,10,736,57,2,-1
6,10075,0,ガチヤグラ,bravo4,1,4,9,1206,60,4,-1
8,10076,0,ガチヤグラ,alpha1,2,5,9,1340,34,5,-1
9,10076,0,ガチヤグラ,alpha2,2,11,10,644,57,1,-1
10,10076,0,ガチヤグラ,alpha3,0,8,7,862,95,2,-1


### 不要な行の削除

In [18]:
# 数値でないデータを含む行を特定する関数
def is_not_numeric(x):
    if pd.isna(x):  # NaNなら数値ではないと判断
        return True
    try:
        float(x)  # 数値に変換できるか試す
        return False  # 変換できたら数値
    except ValueError:
        return True  # 変換できなかったら数値でない

# 各列が数値でないかどうかを判定
non_numeric = long_data_dropped[
    ["win", "Assist", "Death", "Kill", "PaintPoint", "Rank", "Special", "Udemae"]
].applymap(is_not_numeric)

# 数値でないデータを含む行を抽出
non_numeric_rows = long_data_dropped[non_numeric.any(axis=1)]

# 結果を表示
non_numeric_rows.head()

stat,no,win,gameRule,player_type,Assist,Death,Kill,PaintPoint,Rank,Special,Udemae
586,10158,0,ガチホコバトル,alpha3,,,,,,,
590,10158,0,ガチホコバトル,bravo4,,,,,,,
634,10164,1,ガチアサリ,alpha3,,,,,,,
638,10164,1,ガチアサリ,bravo4,,,,,,,
642,10165,0,ガチホコバトル,alpha3,,,,,,,


In [19]:
# non_numeric_rows のインデックスを取得して long_data_dropped から削除
long_data_cleaned = long_data_dropped.drop(non_numeric_rows.index)

# 結果を確認
print(long_data_dropped.shape) # 元のデータの形
print(non_numeric_rows.shape) # 間違ったデータの形
print(long_data_cleaned.shape) # 現在のデータの形

(17395, 11)
(294, 11)
(17101, 11)
